In [25]:
import numpy as np
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

from fealpy.decorator import cartesian
from fealpy.mesh import TriangleMesh
from fealpy.functionspace import LagrangeFiniteElementSpace
from fealpy.boundarycondition import DirichletBC

class pde():
    def __init__(self, mu=1, lam=1):
        self.mu  = mu
        self.lam = lam
        
    def domain(self):
        return [0, 1, 0, 1]
    
    def init_mesh(self, n=1, meshtype='tri'):
        node = np.array([
            (0,0),
            (1,0),
            (1,1),
            (0,1)], dtype=np.float64)
        cell = np.array([(1,2,0), (3,0,2)], dtype=np.int64)
        mesh = TriangleMesh(node, cell)
        mesh.uniform_refine(n)
        return mesh
    
    @cartesian
    def source(self, p):
        x   = p[..., 0]
        y   = p[..., 1]
        mu  = self.mu
        lam = self.lam
        
        sin = np.sin
        cos = np.cos
        val = np.zeros(p.shape, dtype=np.float64)
        
        val[..., 0] = -((2 * mu + lam) * y * (y - 1) * (2 * cos(x) - (x - 1) * sin(x))
                        + (mu + lam) * (2 * x - 1) * (sin(y) + (y - 1) * cos(y)) 
                        + 2 * mu * (x -1) * sin(x))
        val[..., 1] = -((2 * mu + lam) * x * (x - 1) * (2 * cos(y) - (y - 1) * sin(y))
                        + (mu + lam) * (2 * y - 1) * (sin(x) + (x - 1) * cos(x))
                        + 2 * mu * (y - 1) * sin(y))

        return val
    
    def dirichlet(self, p):
        var = np.zeros_like(p)
        return var
    
    def solution(self, p):
        x = p[..., 0]
        y = p[..., 1]
        
        val = np.zeros(p.shape, dtype=np.float64)
        
        val[..., 0] = y * (x - 1) * (y - 1) * np.sin(x)
        val[..., 1] = x * (x - 1) * (y - 1) * np.sin(y)
        return val
        
    def is_dirichlet_boundary(self, p):
        x = p[..., 0]
        y = p[..., 1]
        flag = np.max(np.abs(y)) < 1e-13 
        return flag
    
def error(u, uh):
    e = u - uh
    emax = np.max(np.abs(e))
    return emax
    
pde = pde(0.5, 0.5)
mesh = pde.init_mesh(2)
NN = mesh.number_of_nodes()
NC = mesh.number_of_cells()
#[NC,2] 剖分点及其编号(下标)
node = mesh.entity('node') 
#[NC,3] 剖分区间及其端点编号
cell = mesh.entity('cell') 
#[NC] 每个单元的面积
cm = mesh.entity_measure()

##[NC,3,2] 每个单元上基函数(三个)对x、y的偏导数
#glam = mesh.grad_lambda()
#GLAM = np.broadcast_to(glam[:, :, None, :], shape=[NC,3,2,2])
#GLAM_T = GLAM.swapaxes(2,3)
#GLAM = GLAM + GLAM_T
#
##[NC,3,3] 单元刚度矩阵第一部分
#S = np.einsum("cnij, cmij, c->cnm", GLAM, GLAM, cm)
#glam_sum = np.einsum("cni->cn", glam)
##[NC,3,3] 单元刚度矩阵第二部分
#M = np.einsum("cn, cm->cnm", glam_sum, glam_sum)
#
#I = np.broadcast_to(cell[:, :, None], shape=S.shape)
#J = np.broadcast_to(cell[:, None, :], shape=S.shape)
#
#S = csr_matrix((S.flat, (I.flat, J.flat)), shape=(NN,NN))
#M = csr_matrix((M.flat, (I.flat, J.flat)), shape=(NN,NN))
#A = pde.mu * S / 2 + pde.lam * M

glam_dim3[0,i] = $ [\frac{\partial \varphi_i}{\partial x}, \frac{\partial \varphi_i}{\partial y}] $

In [26]:
glam_x_y = mesh.grad_lambda()
#print("glam_x_y= ", glam_x_y)
glam_dim4 = np.broadcast_to(glam_x_y[:, :, None, :], shape=(NC,3,2,2)).copy()
glam_dim4[:, :, 0, 1] = 0
glam_dim4[:, :, 1, 0] = 0 #[NC,3,2(每个点两个方向),2(两个偏导)]
#print("glam_dim4= ", glam_dim4)

glam_t = glam_dim4.swapaxes(2,3)
glam_dim4 = glam_dim4 + glam_t
#print("glam_dim4= ", glam_dim4)
glam_dim3 = glam_dim4.reshape(NC,6,2)
#print("glam_dim3= ", glam_dim3)

# [NC,6,6]
S = np.einsum("cni, cmi, c->cnm", glam_dim3, glam_dim3, cm)
# [NC,6]
glam_sum = np.einsum("cni->cn", glam_dim3)
#[NC,6,6] 单元刚度矩阵第二部分
M = np.einsum("cn, cm, c->cnm", glam_sum, glam_sum, cm)

cell_x_y = np.zeros((NC,3,2), dtype=np.float64)
cell_x_y[:,:,0] = 2 * cell
cell_x_y[:,:,1] = 2 * cell + 1
cell_x_y = cell_x_y.reshape(NC,6)
I = np.broadcast_to(cell_x_y[:, :, None], shape=S.shape)
J = np.broadcast_to(cell_x_y[:, None, :], shape=S.shape)

S = csr_matrix((S.flat, (I.flat, J.flat)), shape=(2 * NN,2 * NN))
M = csr_matrix((M.flat, (I.flat, J.flat)), shape=(2 * NN,2 * NN))
A = pde.mu * S / 2 + pde.lam * M
#print("A= ", A.toarray())

In [27]:
#glam_x = glam.copy()
#glam_y = glam.copy()
#glam_x[:, :, 1] = 0
#glam_y[:, :, 0] = 0
#
#GLAM_X   = np.broadcast_to(glam_x[:, :, None, :], shape=[NC,3,2,2])
#GLAM_X_T = GLAM_X.swapaxes(2,3)
#GLAM_X   = GLAM_X + GLAM_X_T
#GLAM_Y   = np.broadcast_to(glam_y[:, :, None, :], shape=[NC,3,2,2])
#GLAM_Y_T = GLAM_Y.swapaxes(2,3)
#GLAM_Y   = GLAM_Y + GLAM_Y_T
#
#S1 = np.einsum("cnij, cmij, c->cnm", GLAM_X, GLAM_X, cm)
#S2 = np.einsum("cnij, cmij, c->cnm", GLAM_Y, GLAM_X, cm)
#
#glam_x_sum = np.einsum("cni->cn", glam_x)
#glam_y_sum = np.einsum("cni->cn", glam_y)
#
#M1 = np.einsum("cn, cm->cnm", glam_x_sum, glam_x_sum)
#M2 = np.einsum("cn, cm->cnm", glam_y_sum, glam_x_sum)
#
#S1 = csr_matrix((S1.flat, (I.flat, J.flat)), shape=(NN,NN))
#M1 = csr_matrix((M1.flat, (I.flat, J.flat)), shape=(NN,NN))
#A1 = 2 * pde.mu * S1 + pde.lam * M1
#
#S2 = csr_matrix((S2.flat, (I.flat, J.flat)), shape=(NN,NN))
#M2 = csr_matrix((M2.flat, (I.flat, J.flat)), shape=(NN,NN))
#A2 = 2 * pde.mu * S2 + pde.lam * M2
#
#A = np.zeros((2 * NN, NN), dtype=np.float64)
##A = csr_matrix(A)
#A[0:NN, :] = A1.toarray()
#A[NN:, :]  = A2.toarray()

In [28]:
#print("glam= ", glam)
#print("glam_x= ", glam_x)
#print("glam_y= ", #glam_y)
#print("GLAM_X= ", GLAM_X)
#print("A1= ", A1.toarray())
#print("A2= ", A2.toarray())
#print("A= ", A)

In [29]:
def is_Bd_Node_flag():
    n = mesh.number_of_nodes()
    isBdNode = np.zeros(n, dtype=bool)
    for i in range(n):
        if (pde.is_dirichlet_boundary(mesh.node[i])):
            isBdNode[i] = True
    return isBdNode

In [30]:
from fealpy.quadrature import TriangleQuadrature

NQ = 2
Q = int(NQ * (NQ + 1) / 2)
qf = TriangleQuadrature(NQ)

#bcs [Q,3]; WS[Q,]
bcs,ws = qf.get_quadrature_points_and_weights()
#phi [Q,3,2]
phi = np.broadcast_to(bcs[:, :, None], shape=[int(Q),3,2])
#[Q,3,2(两个分量),2(对x,y的偏导)]
phi_x_y = np.zeros((Q, 3, 2, 2), dtype=np.float64) 
phi_x_y[:, :, :, 0] = phi
phi_x_y[:, :, :, 1] = phi
phi_x_y[:, :, 0, 1] = 0
phi_x_y[:, :, 1, 0] = 0
phi_x_y = phi_x_y.reshape(Q,6,2)

#ps [Q,NC,2]
ps = np.einsum('qi, cim->qcm', bcs, node[cell])
#val [Q,NC,2]
val = pde.source(ps)
#bb [NC,3]
bb = np.einsum('q, qcj, qij, c->ci', ws, val, phi_x_y, cm)
#print("cell= ", cell)
#print("bb= ", bb)
cell_x_y = np.broadcast_to(cell[:,:,None], shape=(NC, 3, 2)).copy()
#print("cell_x_y= ", cell_x_y)
cell_x_y[:,:,0] = 2 * cell_x_y[:,:,0]
cell_x_y[:,:,1] = 2 * cell_x_y[:,:,1] + 1
#print("cell_x_y= ", cell_x_y)
cell_x_y = cell_x_y.reshape(NC, 6)
#print("cel_x_y= ", cell_x_y)

F = np.zeros(2 * NN)
np.add.at(F, cell_x_y, bb)
#print("F= ", F)

isBdNode = mesh.ds.boundary_node_flag()
isInterNode = ~isBdNode
#print("isInterNode= ", isInterNode)
isInterNode = np.broadcast_to(isInterNode[:, None], shape=(NN, 2))
isInterNode = isInterNode.reshape(2 * NN)
#print("isInterNode= ", isInterNode)

uh = np.zeros((2 * NN), dtype=np.float64)
uh[isInterNode] = spsolve(A[:, isInterNode][isInterNode], F[isInterNode])
#print("uh= ", uh)
uh = uh.reshape(NN, 2)
#print("uh= ", uh)

In [31]:
u = pde.solution(node)
e = error(u, uh)
print("e= ", e)
print("u= ", u)
print("uh= ", uh)

e=  0.019235063377807997
u=  [[ 0.          0.        ]
 [-0.         -0.        ]
 [ 0.          0.        ]
 [-0.         -0.        ]
 [ 0.          0.        ]
 [ 0.05992819  0.05992819]
 [ 0.          0.        ]
 [-0.         -0.        ]
 [-0.         -0.        ]
 [ 0.          0.        ]
 [ 0.03479118  0.03479118]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [-0.         -0.        ]
 [ 0.03195182  0.03195182]
 [-0.         -0.        ]
 [-0.         -0.        ]
 [ 0.          0.        ]
 [-0.         -0.        ]
 [ 0.04494614  0.04638824]
 [ 0.03195182  0.03479118]
 [ 0.04638824  0.04494614]
 [ 0.04260242  0.04494614]
 [ 0.04494614  0.04260242]
 [ 0.03479118  0.03195182]]
uh=  [[0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.04069313 0.04069313]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.02579863 0.02579863]
 [0.         0.

In [32]:
#print("Q= ", Q)
#print("phi= ", phi)
#print("phi_x_y= ", phi_x_y)
#print("bb= ", bb)
#print("F= ", F)
#print("isBdNode= ", isBdNode)#

In [33]:
#u = pde.solution(node)
#uh_ = np.broadcast_to(uh[:,None], shape=(NN,2))
#e = error(u, uh_)#
#
#print('u= ', u)
#print('uh= ', uh_)
#print('e= ', e)

In [34]:
#fig = plt.figure()
#ax = fig.gca()
#mesh.add_plot(ax)
#mesh.find_node(ax,showindex=True)
#mesh.find_cell(ax,showindex=True)

#print("NN= ", NN)
#print("NC= ", NC)
#print("glam= \n", glam)
#print("GLAM= ", GLAM)
#print("GLAM.T", GLAM_T)
#print("S= ", S.toarray())
#print("glam_sum= \n", glam_sum)
#print("M= ", M.toarray())
#print("node= ", node)
#print("cell= ", cell)
#print("I= ", I)
#print("J= ", J)

In [35]:
#scale = 10
#mesh.node[:,0] += scale*uh
#mesh.node[:,1] += scale*uh
#mesh.add_plot(plt)
#plt.show()

In [36]:
#print("bcs= ", bcs)
#print("phi= ", phi)
#print("ws= ", ws)
#print("ps= ", ps)
#print("val= ", val)
#print("node[cell]= ", node[cell])
#print("bb= ", bb)
#print("F= ", F)
#print("uh= ", uh)